# Method of Simulated Moments (MSM)

In [2]:
import copy
import respy as rp
import numpy as np
import pandas as pd 

from method_of_simulated_moments import msm
from method_of_simulated_moments import get_moment_vectors
from method_of_simulated_moments import get_diag_weighting_matrix

This notebook contains a step by step tutorial to simulated method of moments estimation using respy. The loss function `msm` requires the following inputs:

    1. params
    2. options
    3. empirical_moments
    4. calc_moments
    5. replace_nans
    6. weighting matrix
    7. return_scalar

**Computing a weighting matrix:**

For the msm estimation, the user has to define a weighting matrix. `get_diag_weighting_matrix` allows the user to automatically create a diagonal weighting matrix that will match the moment vectors used for estimation. The function requires as inputs the *empirical_moments* that are also used in `msm` and a set of weights that are of the same form as *empirical_moments*. If no weights are specified, the function will return the identity matrix. 

## Introductory Example

The msm function requires a number of inputs which will be discussed in the following.

### Arguments

#### The *params* and *options* Arguments

The first step to msm estimation is the simulation of data using a specified model. Respy simulates data using a vector of parameters *params*, which will be the variable of interest for estimation, and a set of *options* that help define the underlying model.

Respy provides a number of example models. For this tutorial we will be using the parameterization from Keane and Wolpin (1994).

In [4]:
params, options, df_emp = rp.get_example_model("kw_94_one")

In [5]:
params

value  \
category            name                                  
delta               delta                        0.9500   
wage_a              constant                     9.2100   
                    exp_edu                      0.0380   
                    exp_a                        0.0330   
                    exp_a_square                -0.0005   
                    exp_b                        0.0000   
                    exp_b_square                 0.0000   
wage_b              constant                     8.4800   
                    exp_edu                      0.0700   
                    exp_b                        0.0670   
                    exp_b_square                -0.0010   
                    exp_a                        0.0220   
                    exp_a_square                -0.0005   
nonpec_edu          constant                     0.0000   
                    at_least_twelve_exp_edu      0.0000   
                    not_edu_last_period      -4000.0000   
nonpec_home         constant                 17750.0000   
shocks_sdcorr       sd_a                         0.2000   
                    sd_b                         0.2500   
                    sd_edu                    1500.0000   
                    sd_home                   1500.0000   
                    corr_b_a                     0.0000   
                    corr_edu_a                   0.0000   
                    corr_edu_b                   0.0000   
                    corr_home_a                  0.0000   
                    corr_home_b                  0.0000   
                    corr_home_edu                0.0000   
lagged_choice_1_edu edu_ten                      1.0000   
initial_exp_edu     10                           1.0000   
maximum_exp         edu                         20.0000   

                                                                                       comment  
category            name                                                                        
delta               delta                                                      discount factor  
wage_a              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_a                                     return to same sector experience  
                    exp_a_square                   return to same sector, quadratic experience  
                    exp_b                                    return to other sector experience  
                    exp_b_square                  return to other sector, quadratic experience  
wage_b              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_b                                     return to same sector experience  
                    exp_b_square                   return to same sector, quadratic experience  
                    exp_a                                    return to other sector experience  
                    exp_a_square                  return to other sector, quadratic experience  
nonpec_edu          constant                            constant reward for choosing education  
                    at_least_twelve_exp_edu        reward for going to college (tuition, etc.)  
                    not_edu_last_period                        reward for going back to school  
nonpec_home         constant                         constant reward of non-market alternative  
shocks_sdcorr       sd_a                     Element 1,1 of standard-deviation/correlation ...  
                    sd_b                     Element 2,2 of standard-deviation/correlation ...  
                    sd_edu                   Element 3,3 of standard-deviation/correlation ...  
                    sd_home

In [6]:
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 1,
 'monte_carlo_sequence': 'random',
 'core_state_space_filters': ["period > 0 and exp_{i} == period and lagged_choice_1 != '{i}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{j}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{k}' and exp_{k} == 0",
  "period == 0 and lagged_choice_1 == '{k}'"],
 'covariates': {'constant': '1',
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2',
  'at_least_twelve_exp_edu': 'exp_edu >= 12',
  'not_edu_last_period': "lagged_choice_1 != 'edu'",
  'edu_ten': 'exp_edu == 10'}}

#### The *calc_moments* Argument

The *calc_moments* argument is the function that should be used within `msm` to calculate moments from the simulated data. It can also be specified as a list or dictionary of multiple functions if different sets of moments should be calculated from different functions.

In this case we will calculate two sets of moments: choice frequencies and parameters that characterize the wage distribution. The moments are saved to a a pandas. DataFrame with time periods as the index and the moments as columns.

In [7]:
def calc_moments(df):
    choices = df.groupby("Period").Choice.value_counts(normalize=True).unstack()
    wages = df.groupby(['Period'])['Wage'].describe()[['mean', 'std']]
    
    return pd.concat([choices, wages], axis=1)

####  The *replace_nans* Argument

Next we define *replace_nans* is a function or list of functions that define how to handle missings in the data. 

In [8]:
def fill_nans_zero(df):
    return df.fillna(0)

#### The *empirical_moments* Argument

The empirical moments are the moments that are calculated from the observed data which the simulated moments should be matched to. The *empirical_moments* argument requires pandas.DataFrames or pandas.Series as inputs. Alternatively, users can input lists or dictionaries containing DataFrames or Series as items. It is necessary, that *calc_moments*, *replace_nans* and *empirical_moments* correspond to each other i.e. *calc_moments* should output moments that are of the same structure as *empirical_moments*.

For this example we calculate the empirical moments the same way that we calculate the simulated moments, so we can be sure that this condition is fulfilled. 

In [9]:
empirical_moments = calc_moments(df_emp)
empirical_moments = fill_nans_zero(empirical_moments)

In [10]:
empirical_moments.tail()

,a,b,edu,home,mean,std
Period,,,,,,
35,0.279,0.703,0.0,0.018,37231.160871,13217.343152
36,0.278,0.702,0.0,0.020,37684.817644,12693.606573
37,0.278,0.704,0.0,0.018,37768.261756,13016.135077
38,0.278,0.710,0.0,0.012,38089.289084,13314.644165
39,0.274,0.708,0.0,0.018,38176.116018,13567.728207


#### The *weighting_matrix* Argument

In [11]:
weighting_matrix = get_diag_weighting_matrix(empirical_moments)

In [12]:
pd.DataFrame(weighting_matrix)

,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


#### The *return_scalar* Argument

The *return_scalar* argument allows us to return the moment errors in vector form. `msm` will return the moment error vector if *return_scalar* is set to **False** and will return the value of the weighted square product of the moment errors if *return_scalar* is set to **True**. 

### MSM Function
We can now compute the msm function. The function will return a value of 0 if we use the true parameter vector as input.

In [13]:
msm(params=params, 
    options=options, 
    calc_moments=calc_moments, 
    replace_nans = fill_nans_zero,
    empirical_moments=empirical_moments, 
    weighting_matrix = weighting_matrix, 
    return_scalar=True
    )

0.0

Using a different parameter vector will result in a value different from 0.

In [14]:
params_sim = params.copy()
params_sim.loc['delta', 'value'] = 0.8

In [15]:
msm(params=params_sim, 
    options=options, 
    calc_moments=calc_moments, 
    replace_nans = fill_nans_zero,
    empirical_moments=empirical_moments, 
    weighting_matrix = weighting_matrix, 
    return_scalar=True
    )

3261729527.4287252

In [16]:
moment_errors = msm(params=params_sim, 
                    options=options, 
                    calc_moments=calc_moments, 
                    replace_nans = fill_nans_zero,
                    empirical_moments=empirical_moments, 
                    weighting_matrix = weighting_matrix, 
                    return_scalar=False
                    )

moment_errors.head()

,0
a_0,0.174
a_1,0.200
a_2,0.207
a_3,0.144
a_4,0.137


## Inputs as Lists or Dictionaries

In the example above we used single elements for all inputs i.e. we used one function to calculate moments, one function to replace missing moments and saved all sets of moments in a single DataFrame. This works well for the example at hand because the inputs are relatively simple, but other applications might require more flexibility. 

For instance, in the example above we use two sets of moments, the choice frequencies in each period and the mean and variance of the wages in each period. Notice how this choice of moments returns one value for each moment in each period which allows us to save all moments in the same DataFrame. However, some applications might use moments that return only one value for multiple periods or deviate from this structure in another way. In this case the moments can't be saved to a single DataFrame. The `msm` functions thus alternatively accepts lists and dictionaries as inputs. This way, different sets of moments can be stored separately. 

Using lists or dictionaries also allows for the use of different replacement rules for different moments. 

### Lists

In [47]:
def calc_choice_freq(df):
    return df.groupby("Period").Choice.value_counts(normalize=True).unstack()

def calc_wage_distr(df):
    return df.groupby(['Period'])['Wage'].describe()[['mean', 'std']]

calc_moments = [calc_choice_freq, calc_wage_distr]

In [48]:
def fill_nans_zero(df):
    return df.fillna(0)

# Combine to list (for now we use the same replacement type for all sets of moments)
replace_nans = [fill_nans_zero, fill_nans_zero]

In [49]:
params, options, df_emp = rp.get_example_model("kw_94_one")
empirical_moments = [calc_choice_freq(df_emp), calc_wage_distr(df_emp)]

empirical_moments[0] = fill_nans_zero(empirical_moments[0])
empirical_moments[1] = fill_nans_zero(empirical_moments[1])

In [50]:
weighting_matrix = get_diag_weighting_matrix(empirical_moments)

In [51]:
msm(params=params, 
    options=options, 
    calc_moments=calc_moments, 
    replace_nans = replace_nans,
    empirical_moments=empirical_moments, 
    weighting_matrix = weighting_matrix, 
    return_scalar=True
    )

0.0

In [52]:
params_sim = params.copy()
params_sim.loc['delta', 'value'] = 0.8

In [53]:
msm(params=params_sim, 
    options=options, 
    calc_moments=calc_moments, 
    replace_nans = replace_nans,
    empirical_moments=empirical_moments, 
    weighting_matrix = weighting_matrix, 
    return_scalar=False
    ).head()

,0
a_0,0.174
a_1,0.200
a_2,0.207
a_3,0.144
a_4,0.137


In [54]:
msm(params=params_sim, 
    options=options, 
    calc_moments=calc_moments, 
    replace_nans = replace_nans,
    empirical_moments=empirical_moments, 
    weighting_matrix = weighting_matrix, 
    return_scalar=True
    )

3261729527.4287252

### Dictionaries

In [35]:
calc_moments = {'Choice Frequencies': calc_choice_freq, 
                'Wage Distribution': calc_wage_distr
                }

In [36]:
replace_nans = {
                'Choice Frequencies': fill_nans_zero, 
                'Wage Distribution': fill_nans_zero
                }

In [37]:
params, options, df_emp = rp.get_example_model("kw_94_one")
choices = calc_choice_freq(df_emp)
choices = fill_nans_zero(choices)
wages = calc_wage_distr(df_emp)
wages = fill_nans_zero(wages)

empirical_moments = {
                     'Choice Frequencies': choices, 
                     'Wage Distribution': wages,
                     }

In [38]:
weighting_matrix = get_diag_weighting_matrix(empirical_moments)

In [45]:
msm(params=params, 
   options=options, 
   calc_moments=calc_moments, 
   replace_nans = replace_nans,
   empirical_moments=empirical_moments, 
   weighting_matrix = weighting_matrix, 
   return_scalar=True
   )

0.0

In [43]:
msm(params=params_sim, 
    options=options, 
    calc_moments=calc_moments, 
    replace_nans = replace_nans,
    empirical_moments=empirical_moments, 
    weighting_matrix = weighting_matrix, 
    return_scalar=False
    ).head()

,0
a_0,0.174
a_1,0.200
a_2,0.207
a_3,0.144
a_4,0.137


In [46]:
msm(params=params_sim, 
    options=options, 
    calc_moments=calc_moments, 
    replace_nans = replace_nans,
    empirical_moments=empirical_moments, 
    weighting_matrix = weighting_matrix, 
    return_scalar=True
    )

3261729527.4287252

## References

> Keane, M. P. and  Wolpin, K. I. (1994). [The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence](https://doi.org/10.2307/2109768). *The Review of Economics and Statistics*, 76(4): 648-672.
